In [33]:
import numpy as np
import os
import os.path as op
import h5py
import shutil
import requests
import statistics

In [34]:
outpath = '../../goesdata/processedData'

In [41]:
def makeLabels(outpath):
    # PATH TO STORE THE TRAINING SAMPLES
    #outpath = '/Users/aleex/DSE/Machine Learning/Project/label/'
    if not os.path.exists(outpath):
        os.makedirs(outpath)

    def genLabel(goes16, goes17):
        n1 = len(goes16)*len(goes16[0])
        n2 = len(goes17)*len(goes17[0])

        std1 = np.std(goes16)
        std2 = np.std(goes17)

        diff = goes16 - goes17
        diff_std = np.std(diff)

        delta = abs(goes16 - goes17)
        label = delta//diff_std
        label = label.astype(int)

        return label

    def writeH5(data, label, uniTime):
        coords = np.array(data).astype(np.float)
        shape = coords.shape
        hdf5_path = outpath + uniTime + '.h5'
        with h5py.File(hdf5_path, mode='w') as f:
            d = f.create_dataset('/data', data = coords)
            l = f.create_dataset('/label', data = label)

    #ROOT URL OF THE NPY FILES
    rootURL = "https://gitlab.com/adomakor412/goesdata/-/raw/master/spring_npy/"
    for idx in range(98, 109):    
        for h in range(0, 24):
            for m in range(0, 6):
                with open('log2', 'a') as log:
                    timeID = f"s2019{str(idx).zfill(3)}{str(h).zfill(2)}{str(m*10).zfill(2)}"
                    
                    goes16_url_07 = rootURL + f"{str(idx).zfill(3)}/OR_ABI-L1b-RadF-M6C07_G16_{timeID}"
                    goes17_url_07 = rootURL + f"{str(idx).zfill(3)}/OR_ABI-L1b-RadF-M6C07_G17_{timeID}"

                    goes16_url_08 = rootURL + f"{str(idx).zfill(3)}/OR_ABI-L1b-RadF-M6C08_G16_{timeID}"
                    goes17_url_08 = rootURL + f"{str(idx).zfill(3)}/OR_ABI-L1b-RadF-M6C08_G17_{timeID}"

                    goes16_url_09 = rootURL + f"{str(idx).zfill(3)}/OR_ABI-L1b-RadF-M6C09_G16_{timeID}"
                    goes17_url_09 = rootURL + f"{str(idx).zfill(3)}/OR_ABI-L1b-RadF-M6C09_G17_{timeID}"
                    
                    goes16_url_10 = rootURL + f"{str(idx).zfill(3)}/OR_ABI-L1b-RadF-M6C10_G16_{timeID}"
                    goes17_url_10 = rootURL + f"{str(idx).zfill(3)}/OR_ABI-L1b-RadF-M6C10_G17_{timeID}"
                    
                   
                    try:
                        response16_07 = requests.get(goes16_url_07, stream=True)
                        response17_07 = requests.get(goes17_url_07, stream=True)

                        response16_08 = requests.get(goes16_url_08, stream=True)
                        response17_08 = requests.get(goes17_url_08, stream=True)

                        response16_09 = requests.get(goes16_url_09, stream=True)
                        response17_09 = requests.get(goes17_url_09, stream=True)

                        response16_10 = requests.get(goes16_url_10, stream=True)
                        response17_10 = requests.get(goes17_url_10, stream=True)

                        print(timeID)
                        
                        with open('temp16_07.npy', 'wb') as temp16:
                            shutil.copyfileobj(response16_07.raw, temp16)
                        with open('temp17_07.npy', 'wb') as temp17:
                            shutil.copyfileobj(response17_07.raw, temp17)
                        with open('temp16_08.npy', 'wb') as temp16:
                            shutil.copyfileobj(response16_08.raw, temp16)
                        with open('temp17_08.npy', 'wb') as temp17:
                            shutil.copyfileobj(response17_08.raw, temp17)
                        with open('temp16_09.npy', 'wb') as temp16:
                            shutil.copyfileobj(response16_09.raw, temp16)
                        with open('temp17_09.npy', 'wb') as temp17:
                            shutil.copyfileobj(response17_09.raw, temp17)
                        with open('temp16_10.npy', 'wb') as temp16:
                            shutil.copyfileobj(response16_10.raw, temp16)
                        with open('temp17_10.npy', 'wb') as temp17:
                            shutil.copyfileobj(response17_10.raw, temp17)
                            
                        data16_07 = np.load('temp16_07.npy')
                        data17_07 = np.load('temp17_07.npy')
                        
                        data16_08 = np.load('temp16_08.npy')
                        data17_08 = np.load('temp17_08.npy')
                        
                        data16_09 = np.load('temp16_09.npy')
                        data17_09 = np.load('temp17_09.npy')
                        
                        data16_10 = np.load('temp16_10.npy')
                        data17_10 = np.load('temp17_10.npy')
                        
                        labels_07 = genLabel(data16_07, data17_07)
                        labels_08 = genLabel(data16_08, data17_08)
                        labels_09 = genLabel(data16_09, data17_09)
                        labels_10 = genLabel(data16_10, data17_10)
                        
                        stack = np.stack([data17_07, data17_08, data17_09, data17_10])
                        stack = stack.ravel
                        labelStack = np.stack([labels_07,labels_08,labels_09,labels_10])
                        vote = np.round(np.median(labelStack,axis=2))
                        writeH5(stack, vote, timeID)
                        
                    except ValueError as e:
                        print("url doesn't exist for " + str(goes16_url))
                        print(e)
                        print('\n')
                        print(str(goes16_url), file=log)
                        print('\n')
                        print(e, file=log)           



In [42]:
makeLabels(outpath)

s20190980000


TypeError: float() argument must be a string or a number, not 'builtin_function_or_method'